# Enterprise Firewall Statistics APIs
 
<p><div class="lev1 toc-item"><a href="#Useful-Snippets" data-toc-modified-id="Useful-Snippets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Useful Snippets</a></div><div class="lev2 toc-item"><a href="#Enable-Debugging" data-toc-modified-id="Enable-Debugging-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Enable Debugging</a></div><div class="lev3 toc-item"><a href="#Pretty-Printing-XML" data-toc-modified-id="Pretty-Printing-XML-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Pretty Printing XML</a></div><div class="lev3 toc-item"><a href="#Enable-netconf" data-toc-modified-id="Enable-netconf-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Enable netconf</a></div><div class="lev1 toc-item"><a href="#Connecting-to-a-Device" data-toc-modified-id="Connecting-to-a-Device-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Connecting to a Device</a></div><div class="lev2 toc-item"><a href="#Capabilities" data-toc-modified-id="Capabilities-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Capabilities</a></div><div class="lev2 toc-item"><a href="#YANG-data-model-Schema" data-toc-modified-id="YANG-data-model-Schema-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>YANG data model Schema</a></div><div class="lev1 toc-item"><a href="#Device-Configuration" data-toc-modified-id="Device-Configuration-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Device Configuration</a></div><div class="lev2 toc-item"><a href="#Getting-the-Configuration" data-toc-modified-id="Getting-the-Configuration-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Getting the Configuration</a></div><div class="lev2 toc-item"><a href="#Using-filters" data-toc-modified-id="Using-filters-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Using filters</a></div><div class="lev2 toc-item"><a href="#Configure-DMVPN-Spoke" data-toc-modified-id="Using-filters-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Configure DMVPN Spoke</a></div><div class="lev2 toc-item"><a href="#Generate-Config-With-Templates" data-toc-modified-id="Generate-Config-With-Templates-331"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Generate Config With Templates</a></div><div class="lev2 toc-item"><a href="#Save-Configuration" data-toc-modified-id="Save-Configuration-332"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Save Configuration</a></div>

# Objective


- <font size="4">How to retrieve the Enterprise Firewall Inspect session counter values</font>
<br><br>
- <font size="4">Store firewall inspect count values in a Time Series Database (TSDB) **Influx DB** </font>
<br><br>
- <font size="4">Plot statistics on to **Grafana** Dashboard </font>

# Resource URL Structure Components

- <font size="4">In order to retrieve firewall statistics either **Inspect count** or **Drop count** we need to use the resource URL `/statistics/fwall/aggregation`</font><br><br>

- <font size="4">**API end point:** `https://vmanage-ip:port/dataservice/statistics/fwall/aggregation` </font><br>
    
![resource-url](images/resource_url.png)

<br>
<font size="4">Now let’s start using the python script to fetch the Enterprise Firewall Inspect sessions count by using below steps</font>

- <font size="4">Login and Authenticate with a vManage instance</font>
<br><br>
- <font size="4">Build query to specify the rules on how to collect enterprise firewall statistics.</font>
<br><br>
- <font size="4">Perform the **POST** operation by sending query in payload</font>


<font size="3"> 
 * [Enterprise Firewall Statistics APIs](#enterprise-firewall-statistics-apis)
     * [Objective](#objective)
     * [Resource URL Structure Components](#resource-url-structure-components-2)
     * [Query Payload](#query-payload)
        * [Example-1](#example-1)
        * [Example-2](#example-2)
        * [Example-3](#example-3)
     * [POST Operation](#post-operation)
     * [Influx DB:](#influx-db)
        * [Connect to InfluxDB](#connect-to-influxdb)
 
## Objective

How to retrieve the firewall inspect counter values, store firewall inspect count values in a Time Series Database (TSDB) Influx DB and plot it on to Grafana Dashboard

## Resource URL Structure Components

In order to retrieve firewall statistics i.e. inspect count, drop count we need to use the resource URL "/statistics/fwall/aggregation"

URL : https://<vmanage-ip:port>/dataservice/statistics/fwall/aggregation

![resource-url](images/resource_url.png)

Now let’s start using the python script to fetch the firewall inspect and drop count by using below steps

  - login and authenticate to a vManage instance
  - Build Query to specify the rules and how to collect firewall statistics. 
  - Perform the POST operation by sending query in payload
</font>

# API Authentication

<font size="4"> 
Let's start by going through Authentication Class which is used to get authenticated with vManage and receive cookie, token which are used for authentication in subsequent API calls.
<br><br>
**API end point:** `https://vmanage-ip:port/j_security_check`
    
Input parameters for Authentication Class:
 - vManage FQDN/IP address
 - vManage port Number
 - Username
 - Password

<br>
On successful POST operation we get **JSESSIONID/Cookie** which is used in subsequent API calls by client. By nature REST API is stateless so we use JSESSIONID for doing session tracking.
</font>

In [8]:
from influxdb import InfluxDBClient
import requests
import sys
import json
import os
import pprint
import time

from requests.packages.urllib3.exceptions import InsecureRequestWarning

vmanage_host = "198.18.1.10"
vmanage_port = "443"
vmanage_username = "admin"
vmanage_password = "admin"

requests.packages.urllib3.disable_warnings()

class Authentication:

    @staticmethod
    def get_jsessionid(vmanage_host, vmanage_port, username, password):
        api = "/j_security_check"
        base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
        url = base_url + api
        payload = {'j_username' : username, 'j_password' : password}
        
        response = requests.post(url=url, data=payload, verify=False)
        try:
            cookies = response.headers["Set-Cookie"]
            jsessionid = cookies.split(";")
            print("\nAuthentication is successful")
            return(jsessionid[0])
        except:
            print("\nAuthentication failed, No valid JSESSION ID returned")
            exit()
       
    @staticmethod
    def get_token(vmanage_host, vmanage_port, jsessionid):
        headers = {'Cookie': jsessionid}
        base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
        api = "/dataservice/client/token"
        url = base_url + api      
        response = requests.get(url=url, headers=headers, verify=False)
        if response.status_code == 200:
            return(response.text)
        else:
            return None


try:
    Auth = Authentication()
    jsessionid = Auth.get_jsessionid(vmanage_host,vmanage_port,vmanage_username,vmanage_password)
    print("\nSession Cookie: ", jsessionid)
    token = Auth.get_token(vmanage_host,vmanage_port,jsessionid)
    print("\nSession Token: ", token)
except Exception as e:
    print(e)


Authentication is successful

Session Cookie:  JSESSIONID=Q0rBX8NrApiVt2z4i6w_f_qiF3E1x8I4M_GtQQRs.5271ea7c-edb1-420b-be9a-4d25756785bd

Session Token:  32789A9351C92817A6737877D9D1C443C8568B03C734740061E6FB4C08F3ED6C16715D57199F69D8FD2207E631FA21133192


# Query Payload

<font size="4"> 
- In order to retrieve the firewall statistics we need to send Query which describes how statistics should be aggregated. 
<br><br>
- Now let’s define the Query which can be used to collect firewall statistics using POST operation on Resource URL `/dataservice/statistics/fwall/aggregation`

**Example-1**

 - Below example query retrieves firewall inspect count values for last 24 hours from all network devices in fabric. <br><br>

 - The aggregation portion of the query determines how data is bucketized/grouped.
<br>

 - Here, the statistics are aggregated in 30 mins intervals across the span of 24 hours.
<br>
    
```
{
"aggregation": {			                # Defines how statistics are bucketized  
"metrics": [
      {
        "property": "fw_total_insp_count",  # Bucketized based on property for example   
        "type": "sum",                        firewall total inspect packet count
        "order": "desc"
      }
    ],
    "histogram": {.                         # Time interval is 30 mins
      "property": "entry_time",
      "type": "minute",
      "interval": 30,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",         # Aggregated data must match both the rules specified below
    "rules": [
      {
        "value": [
          "24"				                Rule #1: Statistics from the last 24 hours
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}

```

**Example-2**

-	Below query retrieves the firewall Drop count for last 24 hours from all network devices in fabric and statistics are aggregated in 30 mins intervals across the span of 24 hours.

```
{
  "aggregation": {
    "metrics": [
      {
        "property": "fw_total_drop_count",
        "type": "sum",
        "order": "desc"
      }
    ],
    "histogram": {
      "property": "entry_time",
      "type": "minute",
      "interval": 30,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",
    "rules": [
      {
        "value": [
          "24"
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}
```

**Example-3**

-	Below query retrieves the firewall Inspect count for last 1 hour and aggregated for 1 minute intervals.

```
{
  "aggregation": {
    "metrics": [
      {
        "property": "fw_total_insp_count",
        "type": "sum",
        "order": "desc"
      }
    ],
    "histogram": {
      "property": "entry_time",
      "type": "minute",
      "interval": 1,
      "order": "asc"
    }
  },
  "query": {
    "condition": "AND",
    "rules": [
      {
        "value": [
          "1"
        ],
        "field": "entry_time",
        "type": "date",
        "operator": "last_n_hours"
      }
    ]
  }
}
```
</font> 

# POST Request

- <font size="4">Using the **Example-1** query let's perform POST request to retrieve the Enterprise Firewall Inspect count statistics for last 24 hours in intervals of 30 mins.</font>
<br><br>
- <font size="4">Now let's have a look at the response to POST operation.</font>
<br><br>

<font size="4">
    
**Sample Response**

- `entry_time` values are in epoch format and correspending key `fw_total_insp_count` contains the Enterprise Firewall Inspect count value at that `entry_time`. 

```
Firewall Inspect count

  "data": [
    {
      "entry_time": 1552807800000,
      "count": 92,
      "fw_total_insp_count": 278
    },
    {
      "entry_time": 1552806000000,
      "count": 360,
      "fw_total_insp_count": 1050
    },
    {
      "entry_time": 1552804200000,
      "count": 360,
      "fw_total_insp_count": 1052
    },
    {
      "entry_time": 1552802400000,
      "count": 360,
      "fw_total_insp_count": 1051
    },
```
</font>

In [34]:
payload = {
            "aggregation": {
            "metrics": [
                            {
                                "property": "fw_total_insp_count",
                                "type": "sum"            
                            }
                       ],
            "field": [
                          {
                                "property": "vdevice_name",
                                "dataType": "string"
                          }
                     ],
            "histogram": 
                          {
                                "property": "entry_time",
                                "type": "minute",
                                "interval": 30,
                                "order": "asc"
                          }
                    },
            "query": 
                    {
                        "condition": "AND",
                        "rules": [
                                    {
                                        "value": ["24"],
                                        "field": "entry_time",
                                        "type": "date",
                                        "operator": "last_n_hours"
                                    }
                                ]
                    }
         }

if token is not None:
    headers = {'Content-Type': "application/json",'Cookie': jsessionid, 'X-XSRF-TOKEN': token}
else:
    headers = {'Content-Type': "application/json",'Cookie': jsessionid}

base_url = "https://%s:%s"%(vmanage_host, vmanage_port)
api_url = "/dataservice/statistics/fwall/aggregation"

url = base_url + api_url

response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)

if response.status_code == 200:

    items = response.json()['data']

    print("\nFirewall inspect session count samples\n",items)

else:
    print("\nFetching firewall inspect session count failed\n")
    print(response.text)


Firewall inspect session count samples
 [{'entry_time': 1578837600000, 'count': 4680, 'vdevice_name': '10.3.0.1', 'fw_total_insp_count': 89}, {'entry_time': 1578837600000, 'count': 4602, 'vdevice_name': '10.3.0.2', 'fw_total_insp_count': 1}, {'entry_time': 1578835800000, 'count': 12116, 'vdevice_name': '10.3.0.1', 'fw_total_insp_count': 160}, {'entry_time': 1578835800000, 'count': 9776, 'vdevice_name': '10.3.0.2', 'fw_total_insp_count': 1}, {'entry_time': 1578834000000, 'count': 13961, 'vdevice_name': '10.3.0.1', 'fw_total_insp_count': 135}, {'entry_time': 1578834000000, 'count': 13260, 'vdevice_name': '10.3.0.2', 'fw_total_insp_count': 3}, {'entry_time': 1578832200000, 'count': 13247, 'vdevice_name': '10.3.0.1', 'fw_total_insp_count': 103}, {'entry_time': 1578832200000, 'count': 13338, 'vdevice_name': '10.3.0.2', 'fw_total_insp_count': 2}, {'entry_time': 1578830400000, 'count': 12480, 'vdevice_name': '10.3.0.1', 'fw_total_insp_count': 95}, {'entry_time': 1578830400000, 'count': 12480

# InfluxDB and Grafana

- <font size="4">InfluxDB is an open-source time series database which is optimized for fast, high-availability storage and retrieval of time series data in fields such as operations monitoring, real-time analytics.</font>
<br><br>
- <font size="4">InfluxDB provides a SQL-like language with built-in time functions for querying a data structure composed of measurements, series, and points</font>
<br><br>
- <font size="4">Grafana is an open-source platform used to build monitoring and analytics dashboards.</font>

<font size="4">Here, we are using InfluxDB as a datasource for Grafana to plot the information on dashboard.</font>


In [35]:
#login credentials for InfluxDB

USER = 'admin'
PASSWORD = 'admin'
DBNAME = 'firewall_inspect'


host='localhost'
port=8086

series = []
total_records = 0

json_body = {}

#loop over the API response variable items and create records to be stored in InfluxDB

for i in items:
    json_body = { "measurement": "firewall_inspect_count",
                  "tags": {
                            "host": i["vdevice_name"],
                          },
                 "time": time.strftime('%m/%d/%Y %H:%M:%S',  time.gmtime(i['entry_time']/1000.)),
                 "fields": {
                            "value": float(i['fw_total_insp_count'])
                }
                }
    series.append(json_body)
    total_records = total_records+1

client = InfluxDBClient(host, port, USER, PASSWORD, DBNAME)

print("Create a retention policy")
retention_policy = 'retention_policy_1'
client.create_retention_policy(retention_policy, '10d', 3, default=True)

print("Write points #: {0}".format(total_records))
client.write_points(series, retention_policy=retention_policy)

Create a retention policy
Write points #: 96


True